In [1]:
using Plots
using CSV
using ControlSystems

gr()

Plots.GRBackend()

In [2]:
#体を揺すって脚を出す歩容
#確定版2019/01/20

zc=0.11 #0.12
g=9.8
dt=0.0025
L=0.15 #0.16
l=0.025 #歩幅
#x=0.03
ft=0.05 #最初の全脚支持期間

#support time
Tsup=Int32(0.4/dt)
#both foot support point time
Tdl=Int32(0.09/dt)
#preview step
M=600
            #r             c         l
real_sx=[(1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L;
    
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l; #第一ZMP
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l;
    
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l; #一度動かない瞬間を作る
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L; 0.0+l; (1/2)*sqrt(3)*L+l;#第二zmp
        (1/2)*sqrt(3)*L; 0.0+l; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L; 0.0+l; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L; 0.0+l; (1/2)*sqrt(3)*L+l;
         
        (1/2)*sqrt(3)*L; 0.0+l; (1/2)*sqrt(3)*L+l;#stop time
    
        (1/2)*sqrt(3)*L+l; 0.0+l; (1/2)*sqrt(3)*L+l;#第3zmp
    
        (1/2)*sqrt(3)*L+l; 0.0+l; (1/2)*sqrt(3)*L+l;
        (1/2)*sqrt(3)*L+l; 0.0+l; (1/2)*sqrt(3)*L+l;
        
        ] #  rcl

real_sy=[0.0; L/2; L;
    
        0.0; L/2; L;
         0.0;  L/2; L;
    
        0.0;  L/2; L; #動かない瞬間を作る
        
        0.0;  L/2; L;
        0.0;  L/2; L;
        0.0;  L/2; L;
        
        0.0;  L/2; L; #sptime
        0.0;  L/2; L;
        
        0.0;  L/2; L;#動かない期間
    
        0.0;  L/2; L;
        0.0;  L/2; L;
        0.0;  L/2; L;

        0.0;  L/2; L;
         ]

pltsx=[(1/2)*sqrt(3)*L; 0.0; (1/2)*sqrt(3)*L] #plot用
pltsy=[0.0; L/2; L]

leg_sign=[0;3;0;0;0;0;2;0;0;0;0;1;0;0;]#上げる脚とステップの関係 #left:3 right:1 center:

#初期姿勢
f_zmp_x=abs(real_sx[1]-real_sx[2])/2
f_zmp_y=L/2+0.00

e_zmp_x=abs(real_sx[1]-real_sx[2])/2+l
e_zmp_y=L/2+0.00 #追加

#Virtual support point
sx=[f_zmp_x
    real_sx[8]+abs(real_sx[4]-real_sx[8])/2+0.005+0.004
    abs(real_sx[1]-real_sx[2])/2+(l/3)
    
    abs(real_sx[1]-real_sx[2])/2+(l/3) #動かない瞬間をつくる
    
    abs(real_sx[1]-real_sx[2])/2+(l/3)+0.05 #sp time
    abs(real_sx[1]-real_sx[2])/2+(l/3)+0.05 #stop time
    real_sx[1]+(l/2)+0.012 #支持脚線の第二ZMP
    abs(real_sx[1]-real_sx[2])/2+(l/3)*2#+0.06
    
    abs(real_sx[1]-real_sx[2])/2+(l/3)*2#+0.05 #stop time
    abs(real_sx[1]-real_sx[2])/2+(l/3)*2 #sp time
    abs(real_sx[1]-real_sx[2])/2+(l/3)*2 #stop time
    
    real_sx[1]/2+l+0.005 #第三zmp
    abs(real_sx[1]-real_sx[2])/2+l+0.012
    
    e_zmp_x]

sy=[f_zmp_y
    abs(real_sy[7]-real_sy[8])/2+0.005+0.003
    L/2
    
    L/2 #一度何もしない瞬間を作る
    
    L/2 #sp time
    L/2#stoptime
    L/2 #ZMP2
    L/2
    
    L/2#stop time
    L/2#sptime
    L/2#stoptime
    
    real_sy[2]+abs(real_sy[2]-real_sy[3])/2-0.004 #第三zmp
    L/2
    
    e_zmp_y]

vpoint_sum=length(sx)
full_leg_step_sum=vpoint_sum-1

#to generate trajectory of ZMP when both foot support point 
x_tdl_a=zeros(Float64,full_leg_step_sum)
x_tdl_b=zeros(Float64,full_leg_step_sum)
y_tdl_a=zeros(Float64,full_leg_step_sum)
y_tdl_b=zeros(Float64,full_leg_step_sum)

for i in 1:full_leg_step_sum
    x_tdl_a[i]=(sx[i]-sx[i+1])/(-Tdl*dt)
    x_tdl_b[i]=sx[i]-x_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

for i in 1:full_leg_step_sum
    y_tdl_a[i]=(sy[i]-sy[i+1])/(-Tdl*dt)
    y_tdl_b[i]=sy[i]-y_tdl_a[i]*(i*Tsup*dt+(i-1)*Tdl*dt)
end

#target ZMP vector
px_ref=zeros(vpoint_sum*Tsup+M+1 + full_leg_step_sum*Tdl)
py_ref=zeros(px_ref)

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum)
        for k in 1:Tsup+M
            px_ref[k+step]=sx[i]
        end
    else
        for k in 1:Tsup
            px_ref[k+step]=sx[i]
        end
        step=step+Tsup
        for s in 1:Tdl
            px_ref[s+step]=x_tdl_a[i]*(s+step)*dt+x_tdl_b[i]
        end
        step=step+Tdl
    end
end

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum)
        for k in 1:Tsup+M
            py_ref[k+step]=sy[i]
        end
    else
        for k in 1:Tsup
            py_ref[k+step]=sy[i]
        end
        step=step+Tsup
        for s in 1:Tdl
            py_ref[s+step]=y_tdl_a[i]*(s+step)*dt+y_tdl_b[i]
        end
        step=step+Tdl
    end
end

#total step
k_max=length(px_ref)-M
#computed ZMP vector
px=zeros(Float64,k_max) 
py=zeros(Float64,k_max);

In [3]:
A=[1 dt dt^2/2
    0 1 dt
    0 0 1]
B=[dt^3/6
    dt^2/2
    dt]
C=[1 0 -zc/g]

Q=[20000 0 0 0
    0 1 0 0
    0 0 1 0
    0 0 0 1]
R=1.0

#state vector
x=[ sx[1]
    0.0
    0.0]

y=[ sy[1]
    0.0
    0.0]

φ=[1 -C*A
    zeros(Float64,3) A]
G=[-C*B
    B]
Gr=[1.0
    0.0
    0.0
    0.0];

In [4]:
k=0

t=[i for i in 1:k_max]
x_plot_log=zeros(Float64,k_max)
x_plot_log[k+1]=x[1] #to record for plot
xv_plot_log=zeros(Float64,k_max)
xv_plot_log[k+1]=x[2]
xa_plot_log=zeros(Float64,k_max)
xa_plot_log[k+1]=x[3]

t=[i for i in 1:k_max]
y_plot_log=zeros(Float64,k_max)
y_plot_log[k+1]=y[1] #to record for plot
yv_plot_log=zeros(Float64,k_max)
yv_plot_log[k+1]=y[2]
ya_plot_log=zeros(Float64,k_max)
ya_plot_log[k+1]=y[3] 

xdu=0
xdR=px_ref[k+1]-px[1] #dR[K+1]
ydu=0
ydR=py_ref[k+1]-py[1] #dR[K+1]

xe=px[1]-C*x
ye=py[1]-C*y

dx=[0; 0; 0]
dy=[0; 0; 0]

X=φ*[xe; dx]+G*xdu+Gr*xdR
Y=φ*[ye; dy]+G*ydu+Gr*ydR
old_x=x
old_y=y
xu=0
yu=0
P=dare(φ,G,Q,R)

ξ=(eye(4)-G*(R+G'*P*G)[1]^(-1)*G'*P)*φ

xsum=0
ysum=0
for j in 1:M
    Fr=-(R+G'*P*G)[1]^(-1)*G'*(ξ')^(j-1)*P*Gr
    xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1+1]) #K=0のみ+1
    ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1+1]) #K=0のみ+1
end
F=-(R+G'*P*G)[1]^(-1)*G'*P*φ
xdu=F*X+xsum
ydu=F*Y+ysum

xu+=xdu
yu+=ydu

1-element Array{Float64,1}:
 0.0

In [5]:
for i in 1:k_max-1
    
    px[k+1]=(C*x)[1]
    py[k+1]=(C*y)[1]
    x=A*x+B*xu[1]
    y=A*y+B*yu[1]    
    k+=1
    x_plot_log[k+1]=x[1] #to record for plot
    xv_plot_log[k+1]=x[2]
    xa_plot_log[k+1]=x[3]
    
    y_plot_log[k+1]=y[1] #to record for plot
    yv_plot_log[k+1]=y[2]
    ya_plot_log[k+1]=y[3]
    dx=x-old_x
    dy=y-old_y
    xdR=px_ref[k+1]-px_ref[k]
    ydR=py_ref[k+1]-py_ref[k]
    xe=px_ref[k]-C*x
    ye=py_ref[k]-C*y    
    X=φ*[xe; dx]+G*xdu[1]+Gr*xdR
    Y=φ*[ye; dy]+G*ydu[1]+Gr*ydR
    old_x=x
    old_y=y
    xsum=0
    ysum=0
    for j in 1:M
        Fr=-(R+G'*P*G)*G'*(ξ')^(j-1)*P*Gr
        xsum +=Fr*(px_ref[k+j]-px_ref[k+j-1])
        ysum +=Fr*(py_ref[k+j]-py_ref[k+j-1])
    end
    F=-(R+G'*P*G)*G'*P*φ
    xdu=F*X+xsum
    ydu=F*Y+ysum
    
    xu+=xdu
    yu+=ydu
end

In [6]:
plot(t*dt,px_ref,lab="Target ZMP",linewidth=3,linecolor="black")
plot!(t*dt,x_plot_log,lab="CoM",linewidth=3,linecolor="blue")
plot!(t*dt,px,lab="ZMP x",linewidth=3,linecolor="red")
plot!(xlab="time[ms]",ylab="ZMP_x[m]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 0.10 
 
 
 0.12 
 
 
 0.14 
 
 
 time[ms] 
 
 
 ZMP 
 
 
 x[m] 
 
<polyline clip-path="url(#clip3102)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,218.934 39.5742,218.934 39.7783,218.934 39.9824,218.934 40.1866,218.934 40.3907,218.934 40.5948,218.934 40.7989,218.934 41.003,218.934 41.2072,218.934 
 41.4113,218.934 41.6154,218.934 41.8195,218.934 42.0236,218.934 42.2278,218.934 42.4319,218.934 42.636,218.934 42.8401,218.934 43.0442,218.934 43.2484,218.934 
 43.4525,218.934 43.6566,218.934 43.8607,218.934 44.0648,218.934 44.2689,218.934 44.4731,218.934 44.6772,218.934 44.8813,218.934 45.0854,218.934 45.2895,218.934 
 45.4937,218.934 45.6978,218.934 45.9019,218.934 46.106,218.934 46.3101,218.934 46.5143,218.934 46.7184,218.934 46.9225,218.934 47.1266,218.934 47.3307,218.934 
 47.5349,218.934 47.739,218.934 47.9431,218.934 48.1472,218.934 48.3513,218.934 48.5555,218.934 48.7596,218.934 48.9637,218.934 49.1678,218.934 49.3719,218.934 
 49.5761,218.934 49.7802,218.934 49.9843,218.934 50.1884,218.934 50.3925,218.934 50.5967,218.934 50.8008,218.934 51.0049,218.934 51.209,218.934 51.4131,218.934 
 51.6173,218.934 51.8214,218.934 52.0255,218.934 52.2296,218.934 52.4337,218.934 52.6379,218.934 52.842,218.934 53.0461,218.934 53.2502,218.934 53.4543,218.934 
 53.6585,218.934 53.8626,218.934 54.0667,218.934 54.2708,218.934 54.4749,218.934 54.679,218.934 54.8832,218.934 55.0873,218.934 55.2914,218.934 55.4955,218.934 
 55.6996,218.934 55.9038,218.934 56.1079,218.934 56.312,218.934 56.5161,218.934 56.7202,218.934 56.9244,218.934 57.1285,218.934 57.3326,218.934 57.5367,218.934 
 57.7408,218.934 57.945,218.934 58.1491,218.934 58.3532,218.934 58.5573,218.934 58.7614,218.934 58.9656,218.934 59.1697,218.934 59.3738,218.934 59.5779,218.934 
 59.782,218.934 59.9862,218.934 60.1903,218.934 60.3944,218.934 60.5985,218.934 60.8026,218.934 61.0068,218.934 61.2109,218.934 61.415,218.934 61.6191,218.934 
 61.8232,218.934 62.0274,218.934 62.2315,218.934 62.4356,218.934 62.6397,218.934 62.8438,218.934 63.048,218.934 63.2521,218.934 63.4562,218.934 63.6603,218.934 
 63.8644,218.934 64.0686,218.934 64.2727,218.934 64.4768,218.934 64.6809,218.934 64.885,218.934 65.0891,218.934 65.2933,218.934 65.4974,218.934 65.7015,218.934 
 65.9056,218.934 66.1097,218.934 66.3139,218.934 66.518,218.934 66.7221,218.934 66.9262,218.934 67.1303,218.934 67.3345,218.934 67.5386,218.934 67.7427,218.934 
 67.9468,218.934 68.1509,218.934 68.3551,218.934 68.5592,218.934 68.7633,218.934 68.9674,218.934 69.1715,218.934 69.3757,218.934 69.5798,218.934 69.7839,218.934 
 69.988,218.934 70.1921,218.934 70.3963,218.934 70.6004,218.934 70.8045,218.934 71.0086,218.934 71.2127,218.934 71.4169,218.934 71.621,218.934 71.8251,218.934 
 72.0292,218.358 72.2333,217.783 72.4375,217.207 72.6416,216.631 72.8457,216.056 73.0498,215.48 73.2539,214.904 73.4581,214.329 73.6622,213.753 73.8663,213.177 
 74.0704,212.602 74.2745,212.026 74.4787,211.45 74.6828,210.874 74.8869,210.299 75.091,209.723 75.2951,209.147 75.4992,208.572 75.7034,207.996 75.9075,207.42 
 76.1116,206.845 76.3157,206.269 76.5198,205.693 76.724,205.117 76.9281,204.542 77.1322,203.966 77.3363,203.39 77.5404,202.815 77.7446,202.239 77.9487,201.663 
 78.1528,201.088 78.3569,200.512 78.561,199.936 78.7652,199.361 78.9693,198.785 79.1734,198.209 79.3775,198.209 79.5816,198.209 79.7858,198.209 79.9899,198.209 
 80.194,198.209 80.3981,198.209 80.6022,198.209 80.8064,198.209 81.0105,198.209 81.2146,198.209 81.4187,198.209 81.6228,198.209 81.827,198.209 82.0311,198.209 
 82.2352,198.209 82.4393,198.209 82.6434,198.209 82.8476,198.209 83.0517,198.209 83.2558,198.209 83.4599,198.209 83.664,198.209 83.8682,198.209 84.0723,198.209 
 84.2764,198.209 84.4805,198.209 84.6846,

In [7]:
plot(t,py_ref,lab="Target ZMP y",linewidth=3,linecolor="black")
plot!(t,y_plot_log,lab="CoM",linewidth=3,linecolor="blue")
plot!(t,py,lab="ZMP y",linewidth=3,linecolor="red")
plot!(xlab="time[ms]",ylab="ZMP[m]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 2000 
 
 
 2500 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 time[ms] 
 
 
 ZMP[m] 
 
<polyline clip-path="url(#clip3202)" style="stroke:#000000; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,122.756 39.5742,122.756 39.7783,122.756 39.9824,122.756 40.1866,122.756 40.3907,122.756 40.5948,122.756 40.7989,122.756 41.003,122.756 41.2072,122.756 
 41.4113,122.756 41.6154,122.756 41.8195,122.756 42.0236,122.756 42.2278,122.756 42.4319,122.756 42.636,122.756 42.8401,122.756 43.0442,122.756 43.2484,122.756 
 43.4525,122.756 43.6566,122.756 43.8607,122.756 44.0648,122.756 44.2689,122.756 44.4731,122.756 44.6772,122.756 44.8813,122.756 45.0854,122.756 45.2895,122.756 
 45.4937,122.756 45.6978,122.756 45.9019,122.756 46.106,122.756 46.3101,122.756 46.5143,122.756 46.7184,122.756 46.9225,122.756 47.1266,122.756 47.3307,122.756 
 47.5349,122.756 47.739,122.756 47.9431,122.756 48.1472,122.756 48.3513,122.756 48.5555,122.756 48.7596,122.756 48.9637,122.756 49.1678,122.756 49.3719,122.756 
 49.5761,122.756 49.7802,122.756 49.9843,122.756 50.1884,122.756 50.3925,122.756 50.5967,122.756 50.8008,122.756 51.0049,122.756 51.209,122.756 51.4131,122.756 
 51.6173,122.756 51.8214,122.756 52.0255,122.756 52.2296,122.756 52.4337,122.756 52.6379,122.756 52.842,122.756 53.0461,122.756 53.2502,122.756 53.4543,122.756 
 53.6585,122.756 53.8626,122.756 54.0667,122.756 54.2708,122.756 54.4749,122.756 54.679,122.756 54.8832,122.756 55.0873,122.756 55.2914,122.756 55.4955,122.756 
 55.6996,122.756 55.9038,122.756 56.1079,122.756 56.312,122.756 56.5161,122.756 56.7202,122.756 56.9244,122.756 57.1285,122.756 57.3326,122.756 57.5367,122.756 
 57.7408,122.756 57.945,122.756 58.1491,122.756 58.3532,122.756 58.5573,122.756 58.7614,122.756 58.9656,122.756 59.1697,122.756 59.3738,122.756 59.5779,122.756 
 59.782,122.756 59.9862,122.756 60.1903,122.756 60.3944,122.756 60.5985,122.756 60.8026,122.756 61.0068,122.756 61.2109,122.756 61.415,122.756 61.6191,122.756 
 61.8232,122.756 62.0274,122.756 62.2315,122.756 62.4356,122.756 62.6397,122.756 62.8438,122.756 63.048,122.756 63.2521,122.756 63.4562,122.756 63.6603,122.756 
 63.8644,122.756 64.0686,122.756 64.2727,122.756 64.4768,122.756 64.6809,122.756 64.885,122.756 65.0891,122.756 65.2933,122.756 65.4974,122.756 65.7015,122.756 
 65.9056,122.756 66.1097,122.756 66.3139,122.756 66.518,122.756 66.7221,122.756 66.9262,122.756 67.1303,122.756 67.3345,122.756 67.5386,122.756 67.7427,122.756 
 67.9468,122.756 68.1509,122.756 68.3551,122.756 68.5592,122.756 68.7633,122.756 68.9674,122.756 69.1715,122.756 69.3757,122.756 69.5798,122.756 69.7839,122.756 
 69.988,122.756 70.1921,122.756 70.3963,122.756 70.6004,122.756 70.8045,122.756 71.0086,122.756 71.2127,122.756 71.4169,122.756 71.621,122.756 71.8251,122.756 
 72.0292,125.441 72.2333,128.126 72.4375,130.811 72.6416,133.496 72.8457,136.181 73.0498,138.866 73.2539,141.551 73.4581,144.236 73.6622,146.921 73.8663,149.606 
 74.0704,152.291 74.2745,154.976 74.4787,157.661 74.6828,160.346 74.8869,163.031 75.091,165.716 75.2951,168.401 75.4992,171.086 75.7034,173.771 75.9075,176.456 
 76.1116,179.141 76.3157,181.826 76.5198,184.511 76.724,187.196 76.9281,189.881 77.1322,192.566 77.3363,195.251 77.5404,197.936 77.7446,200.621 77.9487,203.306 
 78.1528,205.991 78.3569,208.676 78.561,211.362 78.7652,214.047 78.9693,216.732 79.1734,219.417 79.3775,219.417 79.5816,219.417 79.7858,219.417 79.9899,219.417 
 80.194,219.417 80.3981,219.417 80.6022,219.417 80.8064,219.417 81.0105,219.417 81.2146,219.417 81.4187,219.417 81.6228,219.417 81.827,219.417 82.0311,219.417 
 82.2352,219.417 82.4393,219.417 82.6434,219.417 82.8476,219.417 83.0517,219.417 83.2558,219.417 83.4599,219.417 83.664,219.417 83.8682,219.417 84.0723,219.417 
 84.2764,219.417 84.4805,219.417 84.6846,219.417 84.8888,219.417 85.0929,219.417 85

In [8]:
plot(x_plot_log,y_plot_log,lab="CoM",linewidth=1,linecolor="blue")
#plot!(px,py,lab="ZPM",linewidth=1,linecolor="black") #追加
plot!(real_sx,real_sy,st=:scatter, lab="support point")
#plot!(sx,sy,st=:scatter, linecolor="red", lab="target ZPM")
a=[0.15 0.15]
b=[0.19 0.19]
plot!(b,a)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0.075 
 
 
 0.100 
 
 
 0.125 
 
 
 0.150 
 
 
 0.175 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 0.10 
 
 
 0.12 
 
 
 0.14 
 
<polyline clip-path="url(#clip3302)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 233.279,188.189 233.279,188.189 233.279,188.189 233.279,188.189 233.279,188.189 233.279,188.189 233.279,188.189 233.279,188.19 233.279,188.19 233.28,188.191 
 233.28,188.191 233.28,188.192 233.281,188.194 233.281,188.195 233.282,188.198 233.283,188.2 233.284,188.203 233.286,188.207 233.287,188.211 233.289,188.215 
 233.291,188.221 233.293,188.227 233.296,188.234 233.298,188.241 233.302,188.249 233.305,188.259 233.309,188.269 233.313,188.28 233.317,188.291 233.322,188.304 
 233.327,188.318 233.333,188.333 233.339,188.349 233.345,188.366 233.352,188.384 233.359,188.403 233.366,188.423 233.374,188.444 233.383,188.467 233.391,188.491 
 233.401,188.515 233.41,188.542 233.421,188.569 233.431,188.597 233.442,188.627 233.454,188.658 233.466,188.69 233.478,188.724 233.491,188.759 233.505,188.795 
 233.519,188.832 233.533,188.871 233.548,188.911 233.564,188.952 233.579,188.995 233.596,189.039 233.613,189.084 233.63,189.131 233.648,189.179 233.667,189.228 
 233.685,189.279 233.705,189.331 233.725,189.385 233.746,189.44 233.767,189.497 233.788,189.555 233.81,189.614 233.833,189.675 233.856,189.738 233.88,189.802 
 233.905,189.867 233.93,189.934 233.955,190.003 233.982,190.074 234.009,190.146 234.036,190.22 234.064,190.295 234.093,190.372 234.123,190.451 234.153,190.532 
 234.184,190.615 234.215,190.7 234.248,190.787 234.281,190.875 234.315,190.966 234.349,191.059 234.385,191.154 234.421,191.251 234.458,191.351 234.496,191.452 
 234.535,191.557 234.575,191.664 234.615,191.773 234.657,191.885 234.7,191.999 234.744,192.117 234.789,192.237 234.834,192.36 234.882,192.487 234.93,192.616 
 234.979,192.749 235.03,192.885 235.082,193.024 235.135,193.167 235.19,193.313 235.246,193.464 235.303,193.618 235.362,193.776 235.423,193.938 235.485,194.104 
 235.548,194.275 235.613,194.45 235.68,194.629 235.749,194.814 235.819,195.003 235.892,195.197 235.966,195.396 236.042,195.6 236.12,195.81 236.2,196.025 
 236.283,196.246 236.367,196.472 236.453,196.705 236.542,196.943 236.633,197.188 236.727,197.439 236.823,197.696 236.921,197.96 237.022,198.23 237.125,198.507 
 237.231,198.791 237.339,199.082 237.45,199.38 237.564,199.686 237.68,199.999 237.8,200.319 237.922,200.647 238.047,200.982 238.174,201.325 238.305,201.676 
 238.438,202.034 238.575,202.401 238.714,202.775 238.857,203.157 239.002,203.547 239.15,203.945 239.301,204.351 239.455,204.765 239.612,205.186 239.772,205.616 
 239.935,206.053 240.101,206.498 240.27,206.951 240.441,207.411 240.615,207.879 240.792,208.353 240.972,208.835 241.154,209.324 241.339,209.819 241.526,210.321 
 241.715,210.83 241.907,211.344 242.101,211.865 242.297,212.391 242.495,212.922 242.696,213.458 242.897,213.999 243.101,214.545 243.306,215.095 243.513,215.648 
 243.721,216.205 243.93,216.765 244.14,217.328 244.351,217.894 244.564,218.462 244.776,219.031 244.99,219.602 245.203,220.173 245.417,220.745 245.632,221.318 
 245.846,221.89 246.06,222.462 246.274,223.032 246.487,223.602 246.7,224.169 246.912,224.735 247.123,225.298 247.334,225.859 247.543,226.416 247.751,226.97 
 247.958,227.52 248.163,228.065 248.367,228.607 248.569,229.143 248.769,229.675 248.967,230.201 249.164,230.721 249.358,231.236 249.55,231.745 249.74,232.247 
 249.928,232.743 250.113,233.232 250.296,233.714 250.477,234.189 250.654,234.657 250.83,235.117 251.002,235.569 251.172,236.014 251.339,236.452 251.503,236.881 
 251.665,237.302 251.823,237.715 251.979,238.12 252.132,238.517 252.281,238.905 252.428,239.285 252.572,239.657 252.714,240.021 252.852,240.376 252.987,240.723 
 253.119,241.061 253.249,241.392 253.376,241.714 2

In [9]:
function trj_generator(sx,xv,tf,te,dt,legnum)#xの遊脚軌道
    #
    #条件ベクトル
    xf=sx[legnum,tf]
    xe=sx[legnum,te]
    vf=xv[tf]*-1
    ve=xv[te]*-1
    A=[xf; xe; vf; ve]
    
    tf=round(tf*dt,6) #時間に戻す
    te=round(te*dt,6) #第6位で切り捨て
        
    B=[tf^3 tf^2 tf 1
        te^3 te^2 te 1
        3tf^2 2tf 1 0
        3te^2 2te 1 0]

    C=inv(B)*A

    #dt=0.01 #離散時間
    t=[i for i in tf:dt:te] #描画時間
    x=C[1]*.^(t,3)+C[2]*.^(t,2)+C[3]*t+C[4] #位置
    v=3*C[1]*.^(t,2)+2*C[2]*t+C[3] #速度
    a=6*C[1]*t+2*C[2]
    
    return x
    
end

function trj_generator_z(zh,zf,ze,ve,tf,te,dt)
    #support time 0.5
    tf=round(tf*dt,7) #時間に戻す
    te=round(te*dt,7) #第6位で切り捨て
    th=round((te-tf)/2+tf,7)
    
    zh=zf+zh
   
    A_z=[zh; ze; zf; ve]

    B_z=[th^3 th^2 th 1
        te^3 te^2 te 1
        tf^3 tf^2 tf 1
        3te^2 2te 1 0]
    C_z=inv(B_z)*A_z
    
    t=[i for i in tf:dt:te]; #描画時間
     z=C_z[1]*.^(t,3)+C_z[2]*.^(t,2)+C_z[3]*t+C_z[4] #位置
    v=3*C_z[1]*.^(t,2)+2*C_z[2]*t+C_z[3] #速度
    a=6*C_z[1]*t+2*C_z[2]
    return z
end



In [10]:
#脚先位置を重心の座標系に変換する
com_trjx=copy(x_plot_log) #重心位置
com_trjy=copy(y_plot_log)

sx_from_com=zeros(3,length(com_trjx))
sy_from_com=zeros(3,length(com_trjy))   #重心からみた支持脚位置
sz_from_com=[-zc for i in 1:length(com_trjx)*3]
sz_from_com=reshape(sz_from_com,3,length(com_trjx))

sx_from_w=reshape(real_sx,3,vpoint_sum)
sy_from_w=reshape(real_sy,3,vpoint_sum)
vz_end=-0.5 #遊脚が接地する速度 -0.1

step=0
for i in 1:vpoint_sum
    if(i==vpoint_sum) 
        for k in 1:Tsup
            sx_from_com[:,k+step]=sx_from_w[:,i]-com_trjx[k+step]
            sy_from_com[:,k+step]=sy_from_w[:,i]-com_trjy[k+step]
        end
    else
        for k in 1:Tsup
            sx_from_com[:,k+step]=sx_from_w[:,i]-com_trjx[k+step]
            sy_from_com[:,k+step]=sy_from_w[:,i]-com_trjy[k+step]
            
        end
        if(leg_sign[i]!=0)
            #脚を挙げないようにコメントアウト
            sx_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator(sx_from_com,xv_plot_log,1+step,Tsup+step,dt,leg_sign[i])
            sy_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator(sy_from_com,yv_plot_log,1+step,Tsup+step,dt,leg_sign[i])
            sz_from_com[leg_sign[i],1+step:Tsup+step]=trj_generator_z(0.01,-zc+0.01,-zc,vz_end,1+step,Tsup+step,dt)
        end
        step=step+Tsup
    
        for s in 1:Tdl
            sx_from_com[:,s+step]=sx_from_w[:,i]-com_trjx[s+step]
            sy_from_com[:,s+step]=sy_from_w[:,i]-com_trjy[s+step]
        end
        step=step+Tdl
    end
end
;
#-1が使われている可能性あり

In [11]:
function generate_csv(dataframe)
    CSV.write("leg_trj.csv",dataframe,delim=',',header=false)
end

generate_csv (generic function with 1 method)

In [12]:
df = DataFrame(xr=sx_from_com[1,:],
                xc=sx_from_com[2,:],
                xl=sx_from_com[3,:],
                yr=sy_from_com[1,:],
                yc=sy_from_com[2,:],
                yl=sy_from_com[3,:],
                zr=sz_from_com[1,:],
                zc=sz_from_com[2,:],
                zl=sz_from_com[3,:])
df=generate_csv(df)

CSV.Sink(    CSV.Options:
        delim: ','
        quotechar: '"'
        escapechar: '\\'
        null: ""
        dateformat: Base.Dates.DateFormat(Base.Dates.Slot[Base.Dates.DelimitedSlot{Base.Dates.Year}(Base.Dates.Year,'y',4,"-"),Base.Dates.DelimitedSlot{Base.Dates.Month}(Base.Dates.Month,'m',2,"-"),Base.Dates.DelimitedSlot{Base.Dates.Day}(Base.Dates.Day,'d',2,r"(?=\s|$)")],"","english"),IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=0, maxsize=Inf, ptr=1, mark=-1),"leg_trj.csv",0,false,String["xr","xc","xl","yr","yc","yl","zr","zc","zl"],false)